# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/miniconda3/envs/py311/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/py311/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是《易经》中的第一卦，也是八卦之一。乾卦的卦象是由三个阳爻夹一个阴爻构成，象征着天、强、刚、动等特质。乾卦的含义非常丰富，它既象征着阳刚之气、创造力和行动力，也象征着阴柔之美、包容力和适应性。

在《易经》中，乾卦的卦辞是“元、亨、利、贞”，这四个字分别代表了乾卦的四种基本性质。其中，“元”表示至高无上的道德原则，“亨”表示通，表示顺利，“利”表示有益，“贞”表示正。这四个字体现了乾卦所代表的道德品质和行为准则。

乾卦的变化规律是“天行健”，意味着天行不息，运行刚健。乾卦的运行变化表现出一种积极向上、刚健不摧的精神，它告诉我们要积极进取，勇往直前，克服困难，实现目标。

总的来说，乾卦是《易经》中的重要卦象，它反映了宇宙中阳刚之气的运行变化，具有很高的哲学价值和实践意义。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是《易经》中的第五卦，也是八卦之一。讼卦的卦象是由两个阴爻夹一个阳爻构成，象征着经验和智慧。讼卦的含义非常丰富，它既象征着矛盾和争议，也象征着和解和协调。

在《易经》中，讼卦的卦辞是“元、亨、利、贞”，这四个字分别代表了讼卦的四种基本性质。其中，“元”表示至高无上的道德原则，“亨”表示通，表示顺利，“利”表示有益，“贞”表示正。这四个字体现了讼卦所代表的道德品质和行为准则。

讼卦的卦象表示矛盾和争议已经产生，但通过和解和协调，可以实现和谐共处。因此，讼卦教育我们要以智慧和经验来处理矛盾和争议，要寻求和解，避免战争和冲突。

总的来说，讼卦是《易经》中的重要卦象，它反映了宇宙中矛盾和争议的普遍存在，具有很高的哲学价值和实践意义。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [10]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20250813_230857"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [11]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [12]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由两个阳爻（乾为阳）叠加而成，象征着天。它代表了一种刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。', 'content': '\n乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。\n\n在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250813_230857）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [13]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250813_230857）：
[gMASK]sop 周易中的讼卦是什么? 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240225_222843）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [15]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-20250813_230857"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [16]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是八卦之一，由六个阳爻组成，象征着天。它反映了天的刚强和刚正，以及它对世界的控制和统治。乾卦的核心哲学是：君子观此卦象，从而以天为镜， Learn to be峻, Learn to be wide, and Learn to be直。', 'content': '\n乾卦的卦象是由两个阴爻夹一个阳爻构成，象征着天。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [17]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
在周易中，地水师卦是一个由卦象组成的卦名，由上卦坤（地）和下卦坎（水）组成。这一卦象代表着坎卦的变卦，预示着坤卦的力量即将消散，形成坎卦。坎卦本身代表着云，云聚于地，预示着云雨之应，象征君子观此卦象，取舍观于地。

地水师卦的核心象征是云雨之应，意味着谋略得当，得遇帮助，有美好的前景。此卦象征着君子观此卦象，取舍观于地，即君子应当以勤敏治事，忍辱负重，坚持不懈，必有所成。

地水师卦的形成预示着云雨之应，预示着坤卦的力量即将消散，形成坎卦。坎卦本身代表着云，云聚于地，预示着云雨之应，象征君子观此卦象，得舍观于地。

地水师卦的核心象征是云雨之应，此卦象征着勤敏的君子观此卦象，得舍观于地。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


In [18]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么? 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [21]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-20250813_230857"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [22]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由两个阳爻夹一个阴爻构成，象征着天。它代表了一种刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。', 'content': '\n乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。\n\n在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。'}


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [23]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
在周易中，地水师卦是一个卦象，由卦辞、卦象和卦辞的解释构成。卦辞是对卦象的描述，卦象则是周易中卦象的分类，卦辞的解释则是對卦象和卦辞的解释和解读。

地水师卦是由上卦坤（地）下卦坎（水）组成的卦象。坤为地，坎为水，地中有水，象征著地灵验，有预测flexibility，地水相因，相互支持。

根据《周易》的解释，这一卦象代表了一个预测flexibility，适应变化的卦象。得此卦者，需要掌握住地水相因、互为支援的道理，才能获得美满的结果。地水师卦也象征着掌握住灵活适应、因应变化的智慧，才能获得成功。

地水师卦的核心哲学是：地中有水，地水相因，互为支援，象征著地灵验，有预测flexibility，地水相因，相互支持，表示适应变化，灵活应对，互为支援，表示适应变化，灵活应对，互为支援，表示适应变化，灵活应对，从而获得美满的结果。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦 初六 艮山 初六 艮山指险,求吉不利。 官至三台,险至四台。 载至载止,行至行止。 君子乘此卦,必须求得吉利,宜速行止,避免冒险。 得此卦者,应速止行,避免冒险,候时再行。 官至三台,险至四台,载至载止,行至行止。 吉利不断,逢凶化吉,逢险化夷,宜止行。 止于至善,止于至德,止于至道,止于至德。 虽吉但已,虽吉但已,虽吉但已,虽吉但已。 虽吉但已,虽吉但已,虽吉但已,虽吉但已。 虽吉但已,虽吉但已,虽吉但已,虽吉但已。 虽吉但已,虽吉但已,虽吉但已,虽吉但已。 止于至善,止于至德,止于至道,止于至德。 止于至善,止于至德,止于至道,止于至德。 止于至善,止于至德,止于至道,止于至德。 止于至善,止于至德,止于至道,止于至德。 止于至善,止于至德,止于至道,止于至德。 止于至善,止于至德,止于至道,止于至德。 止于至善,止于至德,止于至道,止于至德。 止于至善,止于至德,止于至道,止于至德。 止于至善,止于至德,止于至道,止于至德。 止于至善,止于至德,止于至道,止于至德。 止于至善,止于至德,止于至道,止于至德。 止于至善,止于至德,止于至道,止于至德。 止于至善,止于至德,止于至道,止于至德。 止于至善,止于至德,止于至道,


In [24]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
天水讼卦是一个卦象，由上卦乾（天）和下卦坎（水）组成。在这个卦象中，乾卦代表天，坎卦代表水，两者相互垂直，形成一个卦象。这个卦象预示着天气预报中的天气与实际结果可能会有出入，提醒人们要留意的方向。

关于这个卦象的具体含义，需要结合卦象的构成和卦象 interpreter的解释来综合分析。在风水学中，人们通常会根据卦象来判断家居环境、建筑风水、事业投资等方面的吉凶。在周易学中，卦象的解释则需要结合卦辞和卦象的象征意义来综合分析。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦 坎上坎入 坎宫 坎元 坎正 坎德 坎大 坎小 坎之易 坎之利 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 坎之正 坎之极 坎之和 坎之宜 
